## Calculating Credit Valuation Adjustment (CVA) for OTC derivatives portfolio

This notebook demonstrates how to access portolio from MARVAL and use the async-cva end point of Instrument Pricing Analytics (IPA) in the [LSEG Data Platform](https://developers.lseg.com/en/api-catalog/refinitiv-data-platform/refinitiv-data-platform-apis/documentation#ipa-financial-contracts-cva-definition) to get the Counterparty Valuation Adjustment (CVA) report.

Set up new Portfolio in MARVAL before starting this codebook project:
- Go to [MARVAL Trade Import](https://workspace.refinitiv.com/web/Apps/MARVALTradeImport), use the Excel template included in the CVA codeboook project. You need to download it first.
- Create a new portfolio from MARVAL Trade Import.
- Select Swaps as instrument type, browse your downloaded Excel template and upload all instruments in your portfolio.

#### Learn more

Please visit the [LSEG Developer Community Portal](https://developers.lseg.com/en/api-catalog/refinitiv-data-platform/refinitiv-data-platform-apis/documentation#ipa-financial-contracts-cva-definition) to access the API documentation on CVA and much more.

You may also visit the [API Playground](https://apidocs.refinitiv.com/Apps/ApiDocs) to get more examples, find  end points and download Swagger files

#### Getting Help and Support

If you have any questions regarding the API usage, please post them on the [LSEG Developer Community Q&A Forum](https://community.developers.refinitiv.com/spaces/231/index.html). The LSEG Developer Community will be very pleased to help you. 

## Imports

In [1]:
import pandas as pd
from modules.PortfolioLoader import PortfolioLoader
from modules.EntityManager import CounterpartyManager, CounterpartyParams, SelfEntity
from modules.CVACalculator import CVACalculator, PricingParameters
from modules.CVAVisualizer import CVAVisualizer

## Loading porfolios and instruments

In [ ]:
portfolio_loader = PortfolioLoader()
portfolios = portfolio_loader.show_portfolios()
portfolios

,scope,code,baseCurrency
2,GEDTC-552703,CVASAMPLE,USD


In [3]:
portfolio = portfolio_loader.load_portfolio('CVASAMPLE')
valuation_date = '2024-08-30'

### View portfolio transactions

In [4]:
# if you want to load the transactions as of a date pass the date to the load_instruments function. eg. load_transactions('2024-10-11')
transactions = portfolio_loader.load_transactions(effective_date=valuation_date)
transactions

,Instrument ID,Counterparty
0,LUID_0000AVDH,8589934316
1,LUID_0000AVDJ,8589934333
2,LUID_0000AVDK,4295875726
3,LUID_0000AVDI,8589934316


### Load instruments from MARVAL

In [5]:
instruments = portfolio_loader.load_instruments() 

In [6]:
instruments_brief = pd.DataFrame([{
    'instrumentType': instrument['instrumentType'],
    'csaTag': instrument['csaTag'],
    "instrumentTag": instrument['instrumentDefinition']['instrumentTag']
} for instrument in instruments])

instruments_brief

,instrumentType,csaTag,instrumentTag
0,Swap,CitigroupInc,LUID_0000AVDI
1,Swap,BarclaysPLC,LUID_0000AVDJ
2,Swap,UniCreditSpA,LUID_0000AVDK
3,Swap,CitigroupInc,LUID_0000AVDH


## Loading and associating risk profile to each counterparty

Set credit risk profile on all your counterparties. You can use the [MARVAL Counterparty Editor](https://workspace.refinitiv.com/web/rap/marvalcounterpartyeditor) to search for Credit Curve. Use the CounterpartyManager object below to get current counterparties and set credit risk profiles.

In [7]:
counterparty_manager =  CounterpartyManager()

In [8]:
counterparties = counterparty_manager.get_counterparties(portfolio.counterparties)
counterparties.to_df()

,name,assigned_to
8589934316,CitigroupInc,LUID_0000AVDH
8589934333,BarclaysPLC,LUID_0000AVDJ
4295875726,UniCreditSpA,LUID_0000AVDK


In [9]:
counterparty_manager.enhance_counterparty_data(
            reference_entity="8589934333", 
            enhancement_data = CounterpartyParams(
                                    curve_id="0#BCSBEUAMRBMK=", 
                                    collateral_currency="EUR",
                                    recovery_deal=40, 
                                    recovery_instrument=40
                                    ))

counterparty_manager.enhance_counterparty_data(
            reference_entity="8589934316", 
            enhancement_data = CounterpartyParams(
                                    curve_id="0#CUSAXRBMK=", 
                                    collateral_currency="USD",
                                    recovery_deal=40, 
                                    recovery_instrument=40))

counterparty_manager.enhance_counterparty_data(
            reference_entity="4295875726",
            enhancement_data = CounterpartyParams(
                                    curve_id="0#UNICEUAMRBMK=", 
                                    collateral_currency="EUR",
                                    recovery_deal=40, 
                                    recovery_instrument=40,
                                    ))



In [10]:
counterparties.to_df()

,name,assigned_to,CurveID,CollateralCurrency,RecoveryInstrument,RecoveryDeal
8589934316,CitigroupInc,LUID_0000AVDH,0#CUSAXRBMK=,USD,40,40
8589934333,BarclaysPLC,LUID_0000AVDJ,0#BCSBEUAMRBMK=,EUR,40,40
4295875726,UniCreditSpA,LUID_0000AVDK,0#UNICEUAMRBMK=,EUR,40,40


In [11]:
portfolio_loader.set_counterparties(counterparties)

## CVA Calculation


This sample value an existing portfolio of Swaps. The 'fields' requested are:
- CVA Bilateral and Unilateral (BilateralCvaInReportCcy, UnilateralCvaInReportCcy)
- CVA Bilateral and Unilateral (BilateralDvaInReportCcy, UnilateralDvaInReportCcy)
- MarketValueInReportCcy (MarketValueInReportCcyArray)

Use CVACalculator object below to calculate CVA

In [12]:
cva_calculator = CVACalculator()

##### At this point, Self counterparty should set to value the Debt Valuation Adjustment (DVA)

In [13]:
cva_result = cva_calculator.calculate_cva(
        portfolio=portfolio,
        pricing_parameters = PricingParameters(
                    valuation_date=valuation_date,
                    simulation_count=1000,
                    self_reference_entity="LSEG",
                    self_recovery_rate_percent=40.0,
                    report_ccy="GBP",
                    numeraire_type="Cash"
            ),
        entity = SelfEntity(
                    name="LSEG",
                    collateral_currency="GBP",
                    recovery_instrument=40.0,
                    recovery_deal=40.0,
                    curve_id="0#BBBGBPBMK="
            )
        )

cva_result

,CsaTag,BilateralCvaInReportCcy,BilateralDvaInReportCcy,UnilateralCvaInReportCcy,UnilateralDvaInReportCcy,MarketValueInReportCcyArray,Exposure,PotentialFutureExposure,Allocations,ErrorMessage
0,CitigroupInc,6001.424219,1859.451637,6259.931455,1883.747144,"[111432.49798758417, -1280.1665911302362]","[{'exposureDate': '2024-09-26', 'expectedNegat...","[{'exposureDate': '2024-09-26', 'expectedNegat...","[{'instrumentTag': 'LUID_0000AVDI', 'instrumen...",
1,BarclaysPLC,5604.424684,1992.049134,5740.897828,2020.159791,[814239.7442465454],"[{'exposureDate': '2025-08-29', 'expectedNegat...","[{'exposureDate': '2025-08-29', 'expectedNegat...","[{'instrumentTag': 'LUID_0000AVDJ', 'instrumen...",
2,UniCreditSpA,25010.975676,14194.066685,26376.662865,14402.047031,[229369.77517746927],"[{'exposureDate': '2025-09-08', 'expectedNegat...","[{'exposureDate': '2025-09-08', 'expectedNegat...","[{'instrumentTag': 'LUID_0000AVDK', 'instrumen...",


In [14]:
allocations_df = pd.concat([pd.DataFrame(allocation) for allocation in cva_result['Allocations']], axis=0)
allocations_df = allocations_df.merge(instruments_brief[['instrumentTag', 'csaTag']], on='instrumentTag', how='left')
allocations_df

,instrumentTag,instrumentType,bilateralCvaInReportCcy,bilateralDvaInReportCcy,unilateralCvaInReportCcy,unilateralDvaInReportCcy,csaTag
0,LUID_0000AVDI,Swap,2759.139375,563.023610,2955.209444,583.274943,CitigroupInc
1,LUID_0000AVDH,Swap,3242.284844,1296.420411,3304.722011,1300.472201,CitigroupInc
2,LUID_0000AVDJ,Swap,5604.424684,1992.049134,5740.897828,2020.159791,BarclaysPLC
3,LUID_0000AVDK,Swap,25010.975676,14194.066685,26376.662865,14402.047031,UniCreditSpA


In [16]:
# to save the results to csv run the cell below:
# cva_result.to_csv('cva_result.csv')

In [17]:
# to view the CVA universe, comment out run the function below.
# cva_calculator.cva_universe

In [18]:
# to view the CVA counterparties, comment out run the function below.
# cva_calculator.cva_counterparties

## CVA Visualisation

Use the CVAVisualizer object to view CVA results.

In [19]:
visualizer = CVAVisualizer(cva_result)

#### View allocations for all counterparties

In [20]:
visualizer.plot_allocations()

#### View allocations per counterparty

In [21]:
visualizer.plot_allocations(csa_tag = "CitigroupInc")

#### View market values for all counterparties

In [22]:
visualizer.plot_market_values()

#### View market values per counterparty

In [23]:
visualizer.plot_market_values(csa_tag="CitigroupInc")

#### View Potential Future Exposure

In [24]:
visualizer.plot_potential_future_exposure(csa_tag='CitigroupInc')

#### View Negative and Positive Exposure

In [25]:
visualizer.plot_exposure(csa_tag='CitigroupInc')